## Onnx_conversion

In [31]:
#pip install onnx==1.8.1

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 5.2 MB 3.8 MB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for onnx: filename=onnx-1.8.1-cp36-cp36m-linux_aarch64.whl size=7039324 sha256=da4a002aac40db3f530a686d841cd6729351871015772ec2e3f7427d83c451ba
  Stored in directory: /home/rtml/.cache/pip/wheels/6e/2e/0b/56bf90ac0e0f20a22e645e3e7ca0ab56dcad9b47678694437e
Successfully built onnx
  Attempting uninstall: onnx
    Found existing installation: onnx 1.6.0
    Uninstalling onnx-1.6.0:
      Successfully uninstalled onnx-1.6.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
#Import packages
import random
import torch
from torchvision import transforms
import onnx
from efficientnet_pytorch import EfficientNet 
from torch import nn
from torchviz import make_dot
import onnxruntime as onnxrt
import numpy as np
from onnxsim import simplify

In [2]:
print(onnx.__version__)
print(torch.__version__)

1.8.1
1.10.0


In [3]:
gpuid = 0
device = torch.device('cpu') if gpuid < 0 else torch.device(f'cuda:{gpuid}')
print('Using device:', device)

Using device: cuda:0


## Checking with lift splat shoot

In [6]:
class reduction_5(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate four parameters and assign them as
        member parameters.
        """
        super(reduction_5, self).__init__()
        self.trunk = EfficientNet.from_pretrained("efficientnet-b0")
        self.trunk.set_swish(memory_efficient=False)
    
    def get_eff_depth(self, x):
        #print("cam Encode get_eff_depth")
        # adapted from https://github.com/lukemelas/EfficientNet-PyTorch/blob/master/efficientnet_pytorch/model.py#L231
        endpoints = dict()
        # Stem
        x = self.trunk._swish(self.trunk._bn0(self.trunk._conv_stem(x)))
        prev_x = x

        # Blocks
        for idx, block in enumerate(self.trunk._blocks):
            drop_connect_rate = self.trunk._global_params.drop_connect_rate
            if drop_connect_rate:
                drop_connect_rate *= float(idx) / len(self.trunk._blocks) # scale drop connect_rate
            x = block(x, drop_connect_rate=drop_connect_rate)
            if prev_x.size(2) > x.size(2):
                endpoints['reduction_{}'.format(len(endpoints)+1)] = prev_x
            prev_x = x

        # Head
        endpoints['reduction_{}'.format(len(endpoints)+1)] = x
        x1 = endpoints['reduction_5']
        x2 = endpoints['reduction_4']
        return x1

    def forward(self, x):
        #print("cam Encode forward")
        x1 = self.get_eff_depth(x)

        return x1
reduction_5_model = reduction_5()

Loaded pretrained weights for efficientnet-b0


In [10]:
class reduction_4(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate four parameters and assign them as
        member parameters.
        """
        super(reduction_4, self).__init__()
        
        self.trunk = EfficientNet.from_pretrained("efficientnet-b0")
        self.trunk.set_swish(memory_efficient=False)
    
    def get_eff_depth(self, x):
        #print("cam Encode get_eff_depth")
        # adapted from https://github.com/lukemelas/EfficientNet-PyTorch/blob/master/efficientnet_pytorch/model.py#L231
        endpoints = dict()
        # Stem
        x = self.trunk._swish(self.trunk._bn0(self.trunk._conv_stem(x)))
        prev_x = x

        # Blocks
        for idx, block in enumerate(self.trunk._blocks):
            drop_connect_rate = self.trunk._global_params.drop_connect_rate
            if drop_connect_rate:
                drop_connect_rate *= float(idx) / len(self.trunk._blocks) # scale drop connect_rate
            x = block(x, drop_connect_rate=drop_connect_rate)
            if prev_x.size(2) > x.size(2):
                endpoints['reduction_{}'.format(len(endpoints)+1)] = prev_x
            prev_x = x

        # Head
        endpoints['reduction_{}'.format(len(endpoints)+1)] = x
        x1 = endpoints['reduction_5']
        x2 = endpoints['reduction_4']
        return x2

    def forward(self, x1, x2):
        #print("cam Encode forward")
        x4 = self.get_eff_depth(x1)

        return x4
reduction_4_model = reduction_4()

Loaded pretrained weights for efficientnet-b0


In [11]:
reduction_4_model.to(device)
reduction_5_model.to(device)

reduction_5(
  (trunk): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          32, 8, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          8, 32, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_pr

In [12]:

BATCH_SIZE=1
dummy_input1 = torch.ones(6, 3, 128, 352, device= device)
print(dummy_input1.device)


cuda:0


In [13]:
reduction_4_model.eval()
reduction_5_model.eval()

reduction_5(
  (trunk): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          32, 8, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          8, 32, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_pr

## Creating Onnx 

In [15]:
ONNX_PATH='models/reduction_4_model.onnx'
torch.onnx.export(reduction_4_model, (dummy_input1,dummy_input2), ONNX_PATH,
                    export_params=True,
                    verbose=True, 
                    input_names=['input'],
                    output_names=['output'],

                    opset_version=10,
                    do_constant_folding=False,
                    #example_outputs=torch_out,
                    #strip_doc_string=True,
                    #dynamic_axes=None,
                    keep_initializers_as_inputs=True
                 )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!


graph(%input : Float(6, 3, 128, 352, strides=[135168, 45056, 352, 1], requires_grad=0, device=cuda:0),
      %trunk._blocks.0._se_reduce.weight : Float(8, 32, 1, 1, strides=[32, 1, 1, 1], requires_grad=1, device=cuda:0),
      %trunk._blocks.0._se_reduce.bias : Float(8, strides=[1], requires_grad=1, device=cuda:0),
      %trunk._blocks.0._se_expand.weight : Float(32, 8, 1, 1, strides=[8, 1, 1, 1], requires_grad=1, device=cuda:0),
      %trunk._blocks.0._se_expand.bias : Float(32, strides=[1], requires_grad=1, device=cuda:0),
      %trunk._blocks.1._se_reduce.weight : Float(4, 96, 1, 1, strides=[96, 1, 1, 1], requires_grad=1, device=cuda:0),
      %trunk._blocks.1._se_reduce.bias : Float(4, strides=[1], requires_grad=1, device=cuda:0),
      %trunk._blocks.1._se_expand.weight : Float(96, 4, 1, 1, strides=[4, 1, 1, 1], requires_grad=1, device=cuda:0),
      %trunk._blocks.1._se_expand.bias : Float(96, strides=[1], requires_grad=1, device=cuda:0),
      %trunk._blocks.2._se_reduce.weight 

In [11]:
ONNX_PATH='models/reduction_5_model.onnx'
torch.onnx.export(reduction_5_model, dummy_input1, ONNX_PATH,
                    export_params=True,
                    verbose=True, 
                    input_names=['input'],
                    output_names=['output'],

                    opset_version=10,
                    do_constant_folding=False,
                    #example_outputs=torch_out,
                    #strip_doc_string=True,
                    #dynamic_axes=None,
                    keep_initializers_as_inputs=True
                 )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!


graph(%input : Float(6, 3, 128, 352, strides=[135168, 45056, 352, 1], requires_grad=0, device=cuda:0),
      %trunk._blocks.0._se_reduce.weight : Float(8, 32, 1, 1, strides=[32, 1, 1, 1], requires_grad=1, device=cuda:0),
      %trunk._blocks.0._se_reduce.bias : Float(8, strides=[1], requires_grad=1, device=cuda:0),
      %trunk._blocks.0._se_expand.weight : Float(32, 8, 1, 1, strides=[8, 1, 1, 1], requires_grad=1, device=cuda:0),
      %trunk._blocks.0._se_expand.bias : Float(32, strides=[1], requires_grad=1, device=cuda:0),
      %trunk._blocks.1._se_reduce.weight : Float(4, 96, 1, 1, strides=[96, 1, 1, 1], requires_grad=1, device=cuda:0),
      %trunk._blocks.1._se_reduce.bias : Float(4, strides=[1], requires_grad=1, device=cuda:0),
      %trunk._blocks.1._se_expand.weight : Float(96, 4, 1, 1, strides=[4, 1, 1, 1], requires_grad=1, device=cuda:0),
      %trunk._blocks.1._se_expand.bias : Float(96, strides=[1], requires_grad=1, device=cuda:0),
      %trunk._blocks.2._se_reduce.weight 

In [12]:
y_pred4 = reduction_4_model(dummy_input1)

y_pred5 = reduction_5_model(dummy_input1)
print(y_pred4.shape)
print(y_pred5.shape)
print(f"Model output = {torch.norm(reduction_4_model(dummy_input1) - reduction_4_model(dummy_input1))}")
print(f"Model output = {torch.norm(reduction_5_model(dummy_input1) - reduction_5_model(dummy_input1))}")

torch.Size([6, 112, 8, 22])
torch.Size([6, 320, 4, 11])
Model output = 0.0
Model output = 0.0


## Checking the inference with onnx runtime 

In [14]:
ort_session = onnxrt.InferenceSession('models/reduction_4_model.onnx', None, providers=["CUDAExecutionProvider"])


/home/rtml/.local/lib/python3.6/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:56: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  "Available providers: '{}'".format(name, ", ".join(available_provider_names)))


In [15]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
inputs = [node for node in ort_session.get_inputs()][0]
outputs = [node.name for node in ort_session.get_outputs()]

pred_onx4 = ort_session.run(outputs, {inputs.name: to_numpy(dummy_input1)})
print(pred_onx4[0].shape)
np.testing.assert_allclose(to_numpy(y_pred4.cpu()), pred_onx4[0], rtol=1e-03, atol=1e-05)

(6, 112, 8, 22)


AssertionError: 
Not equal to tolerance rtol=0.001, atol=1e-05

Mismatched elements: 78 / 118272 (0.0659%)
Max absolute difference: 0.00011063
Max relative difference: 0.46363637
 x: array([[[[-17.735939,  -0.434797,   8.992467, ...,  11.1043  ,
           12.302662,  10.307653],
         [ -0.566915,  -7.637162,  -2.469571, ...,   0.45895 ,...
 y: array([[[[-17.735909,  -0.434755,   8.9925  , ...,  11.104317,
           12.302671,  10.307664],
         [ -0.566899,  -7.637188,  -2.469585, ...,   0.458945,...

In [19]:
np.linalg.norm(to_numpy(y_pred4.cpu())-pred_onx4[0])

0.0053324476

In [21]:
ort_session = onnxrt.InferenceSession('models/reduction_5_model.onnx', None, providers=["CUDAExecutionProvider"])


In [22]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
inputs = [node for node in ort_session.get_inputs()][0]
outputs = [node.name for node in ort_session.get_outputs()]

pred_onx5 = ort_session.run(outputs, {inputs.name: to_numpy(dummy_input1)})
print(pred_onx5[0].shape)
np.testing.assert_allclose(to_numpy(y_pred5.cpu()), pred_onx5[0], rtol=1e-03, atol=1e-05)

(6, 320, 4, 11)


In [23]:
np.linalg.norm(to_numpy(y_pred5.cpu())-pred_onx5[0])

0.0016395646

## Checking with onnx simplifier

In [3]:
# load your predefined ONNX model
model = onnx.load('models/camenc_b0.onnx')

# convert model
model_simp, check = simplify(model)

assert check, "Simplified ONNX model could not be validated"

In [4]:
onnx.save(model_simp, 'models/simp_camenc_b0.onnx')

In [5]:
# load your predefined ONNX model
model = onnx.load('models/camenc_b0_1.onnx')

# convert model
model_simp, check = simplify(model)

assert check, "Simplified ONNX model could not be validated"

In [6]:
onnx.save(model_simp, 'models/simp_camenc_b0_1.onnx')